In [2]:
from datetime import datetime, timedelta
import os

from astropy.time import Time, TimeDelta
import numpy as np
import pandas as pd

import db

In [3]:
engine1 = db.get_engine1()
engine2 = db.get_engine2()

In [4]:
info = {}
info['table_name'] = 'm1m3_HardpointActuatorData'
info['expected_rate'] = 50  # Hz
info['output_file'] = 'M1M3_EFD_HardpointData_Insertion_Tracking.csv'
#info['table_name'] = 'MTM1M3TS_thermalData'
#info['expected_rate'] = 1 / 60  # Hz, once per minute
#info['output_file'] = 'M1M3TS_EFD_ThermalData_Insertion_Tracking.csv'
half_time_window = timedelta(minutes=5)
time_increment = timedelta(minutes=10)
query_base = 'select private_sndStamp from {0} where {0}.private_sndStamp between {1} and {2};'

In [9]:
#base_time = datetime(2019, 1, 13, 7, 0, 0)
#now = datetime(2019, 1, 26, 7, 0, 0)
#engine = engine1
#base_time = datetime(2019, 2, 10, 7, 0, 0)
#now = datetime(2019, 2, 12, 7, 0, 0)
#engine = engine2

half_time_window = TimeDelta(300, format='sec')
time_increment = TimeDelta(600, format='sec')
base_time = Time("2019-02-13T18:00:00", scale='utc')
now = Time("2019-02-13T19:00:00", scale='utc')
engine = engine2

timestamp_arr = []
expected_messages_arr = []
received_messages_arr = []
percent_loss_arr = []
inc = 0
while base_time < now:
    #if inc % 5 == 0:
    print(base_time)
    d1 = base_time - half_time_window
    d2 = base_time + half_time_window
    query = query_base.format(info['table_name'], int(d1.unix), int(d2.unix))
    #print(query)
    
    df1 = pd.read_sql_query(query, engine)
    
    dt = d2 - d1
    #print(dt.sec)
    expected_num_messages = round(dt.sec) * info['expected_rate']
    num_received_messages = df1.shape[0]
    delta_messages = expected_num_messages - num_received_messages
    percent_loss = (delta_messages / expected_num_messages) * 100
    
    timestamp_arr.append(base_time)
    expected_messages_arr.append(expected_num_messages)
    received_messages_arr.append(num_received_messages)
    percent_loss_arr.append(percent_loss)
    
    base_time += time_increment
    inc += 1
print("Next Time Start:", base_time)

2019-02-13T18:00:00.000
599.999999999998
2019-02-13T18:10:00.000
599.999999999998
2019-02-13T18:20:00.000
599.999999999998
2019-02-13T18:30:00.000
599.999999999998
2019-02-13T18:40:00.000
599.999999999998
2019-02-13T18:50:00.000
599.999999999998
2019-02-13T19:00:00.000
599.999999999998
Next Time Start: 2019-02-13T19:10:00.000


In [10]:
timestamps = np.array(timestamp_arr)
expected_messages = np.array(expected_messages_arr)
received_messages = np.array(received_messages_arr)
percent_loss = np.array(percent_loss_arr)

In [11]:
output_df = pd.DataFrame({"Expected Messages": expected_messages, 
                          "Received Messages": received_messages, 
                          "Percent Loss": percent_loss}, index=timestamps)

In [12]:
output_file = info['output_file']
header = not os.path.exists(output_file)
output_df.to_csv(output_file, mode='a+', index_label="Timestamps",
                 float_format='%0.4f', header=header)